[toc]

# Pytorch GPU 训练

## 单GPU训练

查看 gpu 是否可用

In [5]:
import torch
torch.cuda.is_available()

False

使用 gpu 训练的时候，有三点需要注意

1. 将模型放到 gpu 上。只需要将最终的模型放到 gpu 上即可，模型中包含的层会被自动放到 gpu 上。
2. 将训练数据放到 gpu 上
3. 将输出数据从gpu上取下来

有两种方式可以将模型放到 gpu 上

1. `model = model.to("cuda:0")`
2. `model = model.cuda()`

注意，上面的模型修改不是原地的，需要进行赋值。

### 使用 tensor.to(device)


In [44]:
import torch
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt

boston = load_boston()
x = torch.Tensor(boston['data'])
y = torch.Tensor(boston['target'])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.nn.Linear(13, 1)
model = model.to(device)

loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 100
for epoch in range(n_epochs):
    x = x.to(device)
    yhat = model(x).view(-1)
    mse = loss(y, yhat)
    
    optimizer.zero_grad()
    mse.backward()
    optimizer.step()

### 使用 tensor.cuda()

In [46]:
import torch
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt

boston = load_boston()
x = torch.Tensor(boston['data'])
y = torch.Tensor(boston['target'])

model = torch.nn.Linear(13, 1)
if torch.cuda.is_available():
    model = model.cuda()

loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 100
for epoch in range(n_epochs):
    if torch.cuda.is_available():
        x = x.cuda()
    yhat = model(x).view(-1)
    mse = loss(y, yhat)
    
    optimizer.zero_grad()
    mse.backward()
    optimizer.step()

# References
1. [(1 封私信 / 18 条消息) 如何在pytorch中正确使用GPU进行训练？ - 知乎](https://www.zhihu.com/question/345418003)